In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

# Libraries

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
stop = set(stopwords.words('english'))
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import json
import ast
import eli5
import shap
from catboost import CatBoostRegressor
from urllib.request import urlopen
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [ ]:
train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')
test = pd.read_csv('../input/tmdb-box-office-prediction/test.csv')

# from this kernel: https://www.kaggle.com/gravix/gradient-in-a-box
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df
        
train = text_to_dict(train)
test = text_to_dict(test)

In [ ]:
train.head()

In [ ]:
train.shape, test.shape

# we have 3000 samples in the train data set and 23 columns (22 features)
# some columns contains list of dictionaries

In [ ]:
# lets extract data from 'belongs_to_collection'
for i, e in enumerate(train['belongs_to_collection'][:5]):
    print(i, e)
    
# we can see some of these are list of dictionaries and some are empty

In [ ]:
# lets calculate how many are empty and non-empty
print(train['belongs_to_collection'].apply(lambda x: type(x) if x != {} else 0).unique())
train['belongs_to_collection'].apply(lambda x: len(x) if x!={} else 0).value_counts()

# there are 2096 values which are empty

In [ ]:
train.columns

In [ ]:
train['collection_name'] = train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x!={} else 0)
train['has_collection'] = train['belongs_to_collection'].apply(lambda x: len(x) if x!={} else 0)
print(train['collection_name'])

test['collection_name'] = test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x!={} else 0)
test['has_collection'] = test['belongs_to_collection'].apply(lambda x: len(x) if x!={} else 0)

train = train.drop(['belongs_to_collection'], axis=1)
test = test.drop(['belongs_to_collection'], axis=1)

In [ ]:
train.shape, test.shape

**Genres**

In [ ]:
for i, e in enumerate(train['genres'][:5]):
    print(i, e)

we will find number of genres in the films, i.e. their count

In [ ]:
print('Number of Genres')

train['genres'].apply(lambda x: len(x) if x!={} else 0).value_counts()

In [ ]:
# all the genres

list_of_genres = list(train['genres'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values)
list_of_genres[:5]

In [ ]:
# generating wordcloud
plt.figure(figsize = (12, 8))
text = ' '.join([i for j in list_of_genres for i in j])

# printing first 100 characters of the text
print(text[:100])

wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False, width=1000, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title('Top Genres')
plt.axis("off")
plt.show()

Drama, Comedy, Thriller are most popular genres

In [ ]:
Counter([i for j in list_of_genres for i in j]).most_common()

Create columns for top 15 most popular genres

In [ ]:
train['num_genres'] = train['genres'].apply(lambda x: len(x) if x!={} else 0)
train['all_genres'] = train['genres'].apply(lambda x: ' '.join(sorted(i['name'] for i in x)) if x!={} else '')
print(train['all_genres'][:5])

top_genres = [m[0] for m in Counter([i for j in list_of_genres for i in j]).most_common(15)]
print(top_genres)

for g in top_genres:
    train['genre_' + g] = train['all_genres'].apply(lambda x: 1 if g in x else 0)
    
# test 
test['num_genres'] = test['genres'].apply(lambda x: len(x) if x!={} else 0)
test['all_genres'] = test['genres'].apply(lambda x: ' '.join(sorted(i['name'] for i in x)) if x!={} else '')
for g in top_genres:
    test['genre_' + g] = test['all_genres'].apply(lambda x: 1 if g in x else 0)
    
train = train.drop(['genres'], axis=1)
test = test.drop(['genres'], axis=1)

In [ ]:
train.shape, test.shape

**Production Companies**

In [ ]:
for i, e in enumerate(train['production_companies'][:5]):
    print(i, e)

In [ ]:
print("Number of production companies in the film")
print("Production companies  ->  no. of films")
train['production_companies'].apply(lambda x: len(x) if x!={} else 0).value_counts()

Most of films have 1-2 production companies, sometimes 3-4. But there are films with 10+ companies! Let's have a look at some of them.

In [ ]:
train[train['production_companies'].apply(lambda x: len(x) if x!={} else 0) > 11]

In [ ]:
list_of_companies = list(train['production_companies'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values)
list_of_companies[:10]

In [ ]:
# 30 most common production companies
Counter([i for j in list_of_companies for i in j]).most_common(30)

In [ ]:
# train
train['num_companies'] = train['production_companies'].apply(lambda x: len(x) if x!={} else 0)
train['all_production_companies'] = train['production_companies'].apply(lambda x: ' '.join(sorted(i['name'] for i in x) if x!={} else ''))
top_companies = [m[0] for m in Counter([i for j in list_of_companies for i in j]).most_common(30)]
for company in top_companies:
    train['production_company_' + company] = train['all_production_companies'].apply(lambda x: 1 if company in x else 0)
    
# test
test['num_companies'] = test['production_companies'].apply(lambda x: len(x) if x!={} else 0)
test['all_production_companies'] = test['production_companies'].apply(lambda x: ' '.join(sorted(i['name'] for i in x) if x!={} else ''))
for company in top_companies:
    test['production_company_' + company] = test['all_production_companies'].apply(lambda x: 1 if company in x else 0)
    
# dropping production_companies column
train = train.drop(['production_companies'], axis=1)
test = test.drop(['production_companies'], axis=1)

In [ ]:
train.shape, test.shape

**Production Countries**

In [ ]:
for i, e in enumerate(train['production_countries'][:5]):
    print(i, e)

In [ ]:
print("Number of production counties in films")
train['production_countries'].apply(lambda x: len(x) if x!={} else 0).value_counts()

so normally the films are produced by one country, but there are films with more than 2 production films

In [ ]:
list_of_countries = list(train['production_countries'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values)
print(list_of_countries[:5])
Counter([i for j in list_of_countries for i in j]).most_common(25)

In [ ]:
# train
train['num_countries'] = train['production_countries'].apply(lambda x: len(x) if x!={} else 0)
train['all_production_countries'] = train['production_countries'].apply(lambda x: ' '.join(sorted(i['name'] for i in x)) if x!={} else '')
top_countries = [m[0] for m in Counter([i for j in list_of_countries for i in j]).most_common(25)]
for country in top_countries:
    train['country_' + country] = train['all_production_countries'].apply(lambda x: 1 if country in x else 0)
    
# test
test['num_countries'] = test['production_countries'].apply(lambda x: len(x) if x!={} else 0)
test['all_production_countries'] = test['production_countries'].apply(lambda x: ' '.join(sorted(i['name'] for i in x)) if x!={} else '')
for country in top_countries:
    test['country_' + country] = test['all_production_countries'].apply(lambda x: 1 if country in x else 0)
    
# dropping production countries column
train = train.drop(['production_countries'], axis=1)
test = test.drop(['production_countries'], axis=1)

In [ ]:
train.shape, test.shape

**Spoken Languages**

In [ ]:
for i, e in enumerate(train['spoken_languages'][:5]):
    print(i, e)

In [ ]:
print('Number of spoken languages in films')
train['spoken_languages'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
list_of_languages = list(train['spoken_languages'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_languages for i in j]).most_common(15)

In [ ]:
train['num_languages'] = train['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
train['all_languages'] = train['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_languages = [m[0] for m in Counter([i for j in list_of_languages for i in j]).most_common(30)]
for g in top_languages:
    train['language_' + g] = train['all_languages'].apply(lambda x: 1 if g in x else 0)
    
test['num_languages'] = test['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
test['all_languages'] = test['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_languages:
    test['language_' + g] = test['all_languages'].apply(lambda x: 1 if g in x else 0)

train = train.drop(['spoken_languages', 'all_languages'], axis=1)
test = test.drop(['spoken_languages', 'all_languages'], axis=1)

In [ ]:
train.shape, test.shape

In [ ]:
for i, e in enumerate(train['Keywords'][:5]):
    print(i, e)

In [ ]:
print('Number of Keywords in films')
train['Keywords'].apply(lambda x: len(x) if x != {} else 0).value_counts().head(10)

In [ ]:
list_of_keywords = list(train['Keywords'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values)
plt.figure(figsize=(16, 12))
text = ' '.join(['_'.join(i.split(' ')) for j in list_of_keywords for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False, width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title("TOP KEYWORDS")
plt.axis("off")
plt.show()

In [ ]:
train['num_Keywords'] = train['Keywords'].apply(lambda x: len(x) if x != {} else 0)
train['all_Keywords'] = train['Keywords'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_keywords = [m[0] for m in Counter([i for j in list_of_keywords for i in j]).most_common(30)]
for g in top_keywords:
    train['keyword_' + g] = train['all_Keywords'].apply(lambda x: 1 if g in x else 0)
    
test['num_Keywords'] = test['Keywords'].apply(lambda x: len(x) if x != {} else 0)
test['all_Keywords'] = test['Keywords'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_keywords:
    test['keyword_' + g] = test['all_Keywords'].apply(lambda x: 1 if g in x else 0)

train = train.drop(['Keywords', 'all_Keywords'], axis=1)
test = test.drop(['Keywords', 'all_Keywords'], axis=1)

In [ ]:
train.shape, test.shape

In [ ]:
for i, e in enumerate(train['cast'][:1]):
    print(i, e)

In [ ]:
print('Number of casted persons in films')
train['cast'].apply(lambda x: len(x) if x != {} else 0).value_counts().head(10)

Those who are casted heavily impact the quality of the film. We have not only the name of the actor, but also the gender and character name/type.

At first let's have a look at the popular names.

In [ ]:
# popular names in the films
list_of_cast_names = list(train['cast'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_cast_names for i in j]).most_common(15)

list of genders

In [ ]:
list_of_cast_genders = list(train['cast'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_cast_genders for i in j]).most_common()

0 is unspecified, 1 is female, and 2 is male.

In [ ]:
list_of_cast_characters = list(train['cast'].apply(lambda x: [i['character'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_cast_characters for i in j]).most_common(15)

it is quite funny the most popular male role is playing himself. :)

In [ ]:
train['num_cast'] = train['cast'].apply(lambda x: len(x) if x != {} else 0)
top_cast_names = [m[0] for m in Counter([i for j in list_of_cast_names for i in j]).most_common(15)]
for g in top_cast_names:
    train['cast_name_' + g] = train['cast'].apply(lambda x: 1 if g in str(x) else 0)
train['genders_0_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
train['genders_1_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
train['genders_2_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))
top_cast_characters = [m[0] for m in Counter([i for j in list_of_cast_characters for i in j]).most_common(15)]
for g in top_cast_characters:
    train['cast_character_' + g] = train['cast'].apply(lambda x: 1 if g in str(x) else 0)
    
test['num_cast'] = test['cast'].apply(lambda x: len(x) if x != {} else 0)
for g in top_cast_names:
    test['cast_name_' + g] = test['cast'].apply(lambda x: 1 if g in str(x) else 0)
test['genders_0_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
test['genders_1_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
test['genders_2_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))
for g in top_cast_characters:
    test['cast_character_' + g] = test['cast'].apply(lambda x: 1 if g in str(x) else 0)

train = train.drop(['cast'], axis=1)
test = test.drop(['cast'], axis=1)

In [ ]:
train.shape, test.shape

crew

In [ ]:
for i, e in enumerate(train['crew'][:1]):
    print(i, e[:10])

In [ ]:
print('Number of casted persons in films')
train['crew'].apply(lambda x: len(x) if x != {} else 0).value_counts().head(10)

The great crew is very important in creating the film. We have not only the names of the crew members, but also the genders, jobs and departments.

At first let's have a look at the popular names.

In [ ]:
list_of_crew_names = list(train['crew'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_crew_names for i in j]).most_common(15)

In [ ]:
list_of_crew_jobs = list(train['crew'].apply(lambda x: [i['job'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_crew_jobs for i in j]).most_common(15)

In [ ]:
list_of_crew_genders = list(train['crew'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_crew_genders for i in j]).most_common(15)

In [ ]:
list_of_crew_departments = list(train['crew'].apply(lambda x: [i['department'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_crew_departments for i in j]).most_common(14)

In [ ]:
list_of_crew_names = train['crew'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values
Counter([i for j in list_of_crew_names for i in j]).most_common(15)

In [ ]:
train['num_crew'] = train['crew'].apply(lambda x: len(x) if x != {} else 0)
top_crew_names = [m[0] for m in Counter([i for j in list_of_crew_names for i in j]).most_common(15)]
for g in top_crew_names:
    train['crew_name_' + g] = train['crew'].apply(lambda x: 1 if g in str(x) else 0)
train['genders_0_crew'] = train['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
train['genders_1_crew'] = train['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
train['genders_2_crew'] = train['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))
top_crew_jobs = [m[0] for m in Counter([i for j in list_of_crew_jobs for i in j]).most_common(15)]
for j in top_crew_jobs:
    train['jobs_' + j] = train['crew'].apply(lambda x: sum([1 for i in x if i['job'] == j]))
top_crew_departments = [m[0] for m in Counter([i for j in list_of_crew_departments for i in j]).most_common(15)]
for j in top_crew_departments:
    train['departments_' + j] = train['crew'].apply(lambda x: sum([1 for i in x if i['department'] == j])) 
    
test['num_crew'] = test['crew'].apply(lambda x: len(x) if x != {} else 0)
for g in top_crew_names:
    test['crew_name_' + g] = test['crew'].apply(lambda x: 1 if g in str(x) else 0)
test['genders_0_crew'] = test['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
test['genders_1_crew'] = test['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
test['genders_2_crew'] = test['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))
for j in top_crew_jobs:
    test['jobs_' + j] = test['crew'].apply(lambda x: sum([1 for i in x if i['job'] == j]))
for j in top_crew_departments:
    test['departments_' + j] = test['crew'].apply(lambda x: sum([1 for i in x if i['department'] == j])) 

train = train.drop(['crew'], axis=1)
test = test.drop(['crew'], axis=1)

In [ ]:
train.shape, test.shape

**Data Exploration**

In [ ]:
train.head()

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))
plt.subplot(1, 2, 1)
plt.hist(train['revenue']);
plt.title('Distribution of revenue');
plt.subplot(1, 2, 2)
plt.hist(np.log1p(train['revenue']));
plt.title('Distribution of log revenue');

As we can see revenue distribution has a high skewness! It is better to use np.log1p of revenue.

In [ ]:
train['log_revenue'] = np.log1p(train['revenue'])

In [ ]:
fig, ax = plt.subplots(figsize = (16, 6))
plt.subplot(1, 2, 1)
plt.hist(train['budget']);
plt.title('Distribution Of Budget');

plt.subplot(1, 2, 2)
plt.hist(np.log1p(train['budget']));
plt.title("Distribution of log of Budget");